0. 导入必要的包

In [ ]:
import lightgbm as lgb
from sklearn.datasets import load_iris
from sklearn.model_selection import StratifiedKFold, GridSearchCV
from sklearn.metrics import roc_auc_score
import numpy as np

1. 加载数据集

In [ ]:
data = load_iris()
X = data.data
y = (data.target != 0).astype(int)  # 将鸢尾花数据集转换为二分类问题

2. 创建 StratifiedKFold 对象（这里使用 10-fold 交叉验证）

In [ ]:
kf = StratifiedKFold(n_splits=10, shuffle=True, random_state=123)

3. LightGBM 参数网格设置

In [ ]:
param_grid = {
    'learning_rate': [0.01, 0.05, 0.1],  # 学习率
    'num_leaves': [31, 50, 100],  # 叶子节点数
    'max_depth': [-1, 10, 20],  # 树的最大深度
    'min_child_samples': [20, 30, 50],  # 一个叶子节点中最小的样本数
    'subsample': [0.8, 0.9, 1.0],  # 数据采样率
    'colsample_bytree': [0.8, 0.9, 1.0],  # 特征采样率
    'reg_alpha': [0, 0.01, 0.1],  # L1 正则化
    'reg_lambda': [0, 0.01, 0.1],  # L2 正则化
    #'scale_pos_weight': [pos_weight]  方法1：手动设置两类数据的权重，在这之前先计算权重：pos_weight = sum(y == 0) / sum(y == 1)
    #'is_unbalance': [True]  方法2：通过is_balance参数自动调整数据平衡
}

4. 定义 LightGBM 分类器

In [ ]:
lgb_estimator = lgb.LGBMClassifier(objective='binary', metric='auc')

5. 使用 GridSearchCV 进行超参数调优和交叉验证

In [ ]:
grid = GridSearchCV(estimator=lgb_estimator, param_grid=param_grid, cv=kf, scoring='roc_auc', verbose=1, n_jobs=-1)
grid.fit(X, y)

6. 输出最佳参数和最佳交叉验证 AUC

In [ ]:
print(f"最佳参数: {grid.best_params_}")
print(f"10-fold 交叉验证的最佳 AUC: {grid.best_score_}")

7. 使用最佳参数重新训练模型

In [ ]:
best_params = grid.best_params_
best_model = lgb.LGBMClassifier(**best_params, objective='binary', metric='auc')

8. 重新训练模型，并用 StratifiedKFold 验证最终结果

In [ ]:
auc_scores = []
for train_index, test_index in kf.split(X, y):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

    # 使用最佳参数训练模型
    best_model.fit(X_train, y_train)

    # 预测概率，用于计算 AUC
    y_pred_proba = best_model.predict_proba(X_test)[:, 1]

    # 计算 AUC
    auc = roc_auc_score(y_test, y_pred_proba)
    auc_scores.append(auc)

9. 输出最终的平均 AUC

In [ ]:
mean_auc = np.mean(auc_scores)
print(f"使用最佳参数进行 10-fold 交叉验证的平均 AUC: {mean_auc:.2f}")

# -------------------------------------------------------------- 分割线 --------------------------------------------------------------

以上就是通过GridSearch搜索最佳超参数，然后基于LightGBM构建二分类预测模型并进行10折交叉验证的代码。在实际建模过程中，我们可能还需要考虑到特征筛选的问题，即我们给不需要用到矩阵中所有的元素来建模，而是根据它们的importance来挑选几个重要的特征然后再根据这些特征选择合适的模型。关于特征筛选可以有几种方式：

1. **通过先验知识来过滤特征。** 例如，我们有一个基因表达矩阵，希望预测病人是否患病。我们已知Wnt这个信号通路与疾病强相关，那我们就可以只用Wnt通路相关的基因来构建表达矩阵进行建模；
2. **通过设定硬阈值来筛选特征。**
3. **通过递归特征消除（RFE）来选择特征。**

首先我们来看手动设定特征importance阈值来筛选特征：

In [ ]:
...
# 7. 获取特征重要性并移除不重要的特征
best_model = grid.best_estimator_
feature_importances = best_model.feature_importances_
print("特征重要性:", feature_importances)

# 设置一个特征重要性阈值，移除不重要的特征，这里的threshold可根据需求修改
threshold = 10
important_features_idx = np.where(feature_importances > threshold)[0]

# 只保留重要特征
X_important = X[:, important_features_idx]

后续代码：

In [ ]:
# 8. 使用仅保留重要特征的数据重新进行交叉验证
auc_scores = []
for train_index, test_index in kf.split(X_important, y):
    X_train, X_test = X_important[train_index], X_important[test_index]
    y_train, y_test = y[train_index], y[test_index]

    # 重新训练模型
    best_model.fit(X_train, y_train)

    # 预测概率，用于计算 AUC
    y_pred_proba = best_model.predict_proba(X_test)[:, 1]

    # 计算 AUC
    auc = roc_auc_score(y_test, y_pred_proba)
    auc_scores.append(auc)

# 9. 输出最终的平均 AUC
mean_auc = np.mean(auc_scores)
print(f"使用重要特征进行 10-fold 交叉验证的平均 AUC: {mean_auc:.2f}")

然后我们来看通过RFE方法来选择特征：

In [ ]:
from sklearn.feature_selection import RFE

...

# 4. 定义 LightGBM 分类器
lgb_estimator = lgb.LGBMClassifier(objective='binary', metric='auc')

# 5. 使用 RFE 进行特征选择
rfe = RFE(lgb_estimator, n_features_to_select=2)  # 设置你想保留的特征数量
rfe.fit(X, y)

# 打印哪些特征被保留
print("选中的特征:", rfe.support_)
print("特征排名:", rfe.ranking_)

# 使用选择后的特征进行交叉验证
X_rfe = rfe.transform(X)

# 8. 使用 RFE 选择后的特征进行交叉验证
auc_scores = []
for train_index, test_index in kf.split(X_rfe, y):
    X_train, X_test = X_rfe[train_index], X_rfe[test_index]
    y_train, y_test = y[train_index], y[test_index]

    # 训练模型
    best_model.fit(X_train, y_train)

    # 预测概率，用于计算 AUC
    y_pred_proba = best_model.predict_proba(X_test)[:, 1]

    # 计算 AUC
    auc = roc_auc_score(y_test, y_pred_proba)
    auc_scores.append(auc)

# 9. 输出最终的平均 AUC
mean_auc = np.mean(auc_scores)
print(f"使用 RFE 进行 10-fold 交叉验证的平均 AUC: {mean_auc:.2f}")